# Data Ingestion

In [4]:
# machine learning classification
from pyforest import *
import datetime, pickle, copy, warnings
import cryptocompare
import requests
import plotly.express as px
import plotly.graph_objects as go
from time import time
from pandas import DataFrame, concat
from sklearn import metrics
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
from math import sqrt
from pandas import DataFrame, concat


apiKey = "xxxxxxxxxxx"
url = "https://min-api.cryptocompare.com/data/histohour"

# BTC 1st 2000 datapoints
payload = {"api_key": apiKey, "fsym": "BTC", "tsym": "USD", "limit": 2000}

result = requests.get(url, params=payload).json()

BitCoin1 = DataFrame(result["Data"])

BitCoin1["time"] = pd.to_datetime(BitCoin1["time"], unit="s")

BitCoin1.set_index("time", inplace=True)

# 2nd 2000 datapoints
payload = {
    "api_key": apiKey,
    "fsym": "BTC",
    "tsym": "USD",
    "limit": 2000,
    "toTs": (1601632800),
}

result = requests.get(url, params=payload).json()

BitCoin2 = DataFrame(result["Data"])

BitCoin2["time"] = pd.to_datetime(BitCoin2["time"], unit="s")

BitCoin2.set_index("time", inplace=True)

# 3rd 2000 datapoints
payload = {
    "api_key": apiKey,
    "fsym": "BTC",
    "tsym": "USD",
    "limit": 2000,
    "toTs": (1593572400),
}

result = requests.get(url, params=payload).json()

BitCoin3 = DataFrame(result["Data"])

BitCoin3["time"] = pd.to_datetime(BitCoin3["time"], unit="s")

BitCoin3.set_index("time", inplace=True)

# 4th 2000 datapoints
payload = {
    "api_key": apiKey,
    "fsym": "BTC",
    "tsym": "USD",
    "limit": 2000,
    "toTs": (1596571200),
}

result = requests.get(url, params=payload).json()

BitCoin4 = DataFrame(result["Data"])

BitCoin4["time"] = pd.to_datetime(BitCoin4["time"], unit="s")

BitCoin4.set_index("time", inplace=True)

# combining all bitcoin data (8000 data points)
combineData1 = BitCoin2.append(BitCoin1)

combineData2 = BitCoin3.append(combineData1)

BitCoin = BitCoin4.append(combineData2)  # final BitCoin dataset

print(BitCoin.tail(2))
# saving btc data set
BitCoin.to_csv("BitCoinRaw.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                         high       low      open  volumefrom     volumeto  \
time                                                                         
2021-02-25 07:00:00  50603.54  50158.30  50503.52     1601.87  80846961.24   
2021-02-25 08:00:00  50559.02  50222.74  50471.50      394.02  19858317.73   

                        close conversionType conversionSymbol  
time                                                           
2021-02-25 07:00:00  50471.50         direct                   
2021-02-25 08:00:00  50402.66         direct                   


# Feature engineering

In [5]:
df = pd.read_csv("BitCoinRaw.csv")
df.set_index("time", inplace=True)

df.drop(columns=["conversionType", "conversionSymbol"], axis=1, inplace=True)

values = DataFrame(df.close.values)
lags = 8
columns = [values]
for i in range(1, (lags + 1)):
    columns.append(values.shift(i))

dt = concat(columns, axis=1)

columns = ["Lag"]
for i in range(1, (lags + 1)):
    columns.append("Lag" + str(i))
dt.columns = columns
dt.index = df.index

finalDataSet = concat([df, dt], axis=1)

finalDataSet.dropna(inplace=True)

finalDataSet["S_10"] = finalDataSet["close"].rolling(window=10).mean()

finalDataSet["Corr"] = (
    finalDataSet["close"].rolling(window=10).corr(finalDataSet["S_10"])
)

finalDataSet["d_20"] = finalDataSet["close"].shift(480)

finalDataSet["5EMA"] = (
    finalDataSet["close"].ewm(span=5, adjust=True, ignore_na=True).mean()
)

finalDataSet["10EMA"] = (
    finalDataSet["close"].ewm(span=10, adjust=True, ignore_na=True).mean()
)

finalDataSet["20EMA"] = (
    finalDataSet["close"].ewm(span=20, adjust=True, ignore_na=True).mean()
)

finalDataSet["mean"] = (finalDataSet["low"] + finalDataSet["high"]) / 2

finalDataSet["returns"] = (
    (finalDataSet["close"] - finalDataSet["open"]) / finalDataSet["open"] * 100.0
)

finalDataSet["volume"] = finalDataSet["volumeto"] - finalDataSet["volumefrom"]

finalDataSet.drop(["volumefrom", "volumeto"], 1, inplace=True)

finalDataSet.dropna(inplace=True)

finalDataSet = finalDataSet.drop(["Lag"], axis=1)

finalDataSet = finalDataSet.astype(float)

finalDataSet = finalDataSet.sort_index(ascending=True)
# dataframe.head(2)

# save data
finalDataSet.to_csv("finalDataSet.csv", header=True)

print(finalDataSet.tail())

<IPython.core.display.Javascript object>

                         high       low      open     close      Lag1  \
time                                                                    
2021-02-25 04:00:00  50396.53  49614.02  50277.51  49652.82  50277.51   
2021-02-25 05:00:00  50461.08  49587.25  49652.82  50432.31  49652.82   
2021-02-25 06:00:00  50613.69  50275.48  50432.31  50503.52  50432.31   
2021-02-25 07:00:00  50603.54  50158.30  50503.52  50471.50  50503.52   
2021-02-25 08:00:00  50559.02  50222.74  50471.50  50402.66  50471.50   

                         Lag2      Lag3      Lag4      Lag5      Lag6  ...  \
time                                                                   ...   
2021-02-25 04:00:00  50684.94  50505.54  50535.37  49738.17  48863.06  ...   
2021-02-25 05:00:00  50277.51  50684.94  50505.54  50535.37  49738.17  ...   
2021-02-25 06:00:00  49652.82  50277.51  50684.94  50505.54  50535.37  ...   
2021-02-25 07:00:00  50432.31  49652.82  50277.51  50684.94  50505.54  ...   
2021-02-25 08:00:00 

## Train-Test data set

In [7]:
import pandas as pd
from pandas import concat, DataFrame
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import sklearn.externals
import joblib

finalDataSet = pd.read_csv("finalDataSet.csv")
finalDataSet.set_index("time", inplace=True)
# print(df.tail())


foreCastColumn = "close"  # creating label

foreCastOut = int(12)  # prediction for next 12 hrs

finalDataSet["label"] = finalDataSet[foreCastColumn].shift(-foreCastOut)

X = np.array(finalDataSet.drop(["label"], axis=1))

# normalize data
X = preprocessing.scale(X)

X_foreCastOut = X[-foreCastOut:]

X = X[:-foreCastOut]

finalDataSet.dropna(inplace=True)

y = np.array(finalDataSet["label"])

C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


## Time-series cross validation

In [8]:
# Split the data into train and test data set
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

## Model development

In [10]:
# regression model
Model = ElasticNet(alpha=0.0001, l1_ratio=0.5, random_state=0).fit(X_train, y_train)

# cross validated accucary on train set
scores = cross_val_score(Model, X_train, y_train, cv=tscv)

print("Training Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Intercept:", Model.intercept_)
print("Slope:", Model.coef_[0])

C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 684446691.0199666, tolerance: 13929743.731469138
  model = cd_fast.enet_coordinate_descent(
C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25024706.176950004, tolerance: 123419.72625487612
  model = cd_fast.enet_coordinate_descent(
C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 35962452.99740169, tolerance: 208018.32587141843
  model = cd_fast.enet_coordinate_descent(
C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWa

Training Accuracy: 0.85 (+/- 0.28)
Intercept: 16309.811601739952
Slope: 8173.083563755378


C:\Users\Sarit\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 68750980.73926553, tolerance: 783644.8973368459
  model = cd_fast.enet_coordinate_descent(


## Accuracy metrics

In [11]:
# prediction on training
trainPredict = Model.predict(X_train)
r_squared = r2_score(y_train, trainPredict)
mae = np.mean(abs(trainPredict - y_train))
rmse = np.sqrt(np.mean((trainPredict - y_train) ** 2))
rae = np.mean(abs(trainPredict - y_train)) / np.mean(abs(y_train - np.mean(y_train)))
rse = np.mean((trainPredict - y_train) ** 2) / np.mean(
    (y_train - np.mean(y_train)) ** 2
)
sumOfDf = DataFrame(
    index=[
        "R-squared",
        "Mean Absolute Error",
        "Root Mean Squared Error",
        "Relative Absolute Error",
        "Relative Squared Error",
    ]
)
sumOfDf["Training metrics"] = [r_squared, mae, rmse, rae, rse]

# prediction of test
testPredict = Model.predict(X_test)
r_squared = r2_score(y_test, testPredict)
mae = np.mean(abs(testPredict - y_test))
rmse = np.sqrt(np.mean((testPredict - y_test) ** 2))
rae = np.mean(abs(testPredict - y_test)) / np.mean(abs(y_test - np.mean(y_test)))
rse = np.mean((testPredict - y_test) ** 2) / np.mean((y_test - np.mean(y_test)) ** 2)

sumOfDf["Validation metrics"] = [r_squared, mae, rmse, rae, rse]
sumOfDf = sumOfDf.round(decimals=3)

print(sumOfDf)  # accuracy check

                         Training metrics  Validation metrics
R-squared                           0.990               0.942
Mean Absolute Error               161.021            1370.179
Root Mean Squared Error           463.578            1797.669
Relative Absolute Error             0.056               0.214
Relative Squared Error              0.010               0.058


## Saving model

In [12]:
# Save model to file in the current working directory
fileName = "ElasticModel.pkl"
joblib.dump(Model, fileName)

# Load from file
ElasticModel = joblib.load(fileName)
# ElasticModel.predict(X_test)
# print(r2_score(y_test, ElasticModel.predict(X_test)))

## Future forecast

In [13]:
# forecast future 12 hrs values
foreCastFutureValues = DataFrame(ElasticModel.predict(X_foreCastOut))
# print(foreCastFutureValues)

# assigning names to columns
foreCastFutureValues.rename(columns={0: "Forecast"}, inplace=True)

newDataframe = finalDataSet.tail(foreCastOut)

newDataframe.reset_index(inplace=True)

# Fixing future datetime
newDataframe = newDataframe.append(
    DataFrame(
        {
            "time": pd.date_range(
                start=newDataframe.time.iloc[-1],
                periods=(len(newDataframe) + 1),
                freq="H",
                closed="right",
            )
        }
    )
)

newDataframe.set_index("time", inplace=True)

newDataframe = newDataframe.tail(foreCastOut)

foreCastFutureValues.index = newDataframe.index

print("12 hours forecast (hourly):")
foreCastFutureValues.reset_index(inplace=True)

print(foreCastFutureValues)

12 hours forecast (hourly):
                  time      Forecast
0  2021-02-24 21:00:00  49809.559723
1  2021-02-24 22:00:00  49683.454960
2  2021-02-24 23:00:00  50486.824487
3  2021-02-25 00:00:00  51455.979472
4  2021-02-25 01:00:00  51587.313733
5  2021-02-25 02:00:00  51616.122748
6  2021-02-25 03:00:00  51378.035669
7  2021-02-25 04:00:00  51043.579239
8  2021-02-25 05:00:00  51207.272646
9  2021-02-25 06:00:00  51419.231345
10 2021-02-25 07:00:00  51383.407690
11 2021-02-25 08:00:00  51347.314168
